In [11]:
import pandas as pd
import numpy as np
import warnings 
from datetime import datetime
warnings.filterwarnings('ignore')

## Read Data

In [62]:
movies = pd.read_csv("../../data/movie/movies.csv")
ratings = pd.read_csv("../../data/movie/ratings.csv")
genres = pd.read_pickle("../../data/movie/genres_dummies.p")

## Preprocessing

In [70]:
ratings=ratings.merge(genres, left_on='movieId',right_index=True)

In [71]:
ratings.head(1)

,userId,movieId,rating,timestamp,(no genres listed),Action,Adventure,Animation,Children,Comedy,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,1,4.0,2000-07-31 03:45:03,0,0,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0


In [104]:
ratings=ratings.replace(0,np.nan)
ratings.head(1)

,userId,movieId,rating,timestamp,(no genres listed),Action,Adventure,Animation,Children,Comedy,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,1,4.0,2000-07-31 03:45:03,NaN,NaN,1.0,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Train test split

In [74]:
from sklearn.model_selection import train_test_split

In [75]:
train, test = train_test_split(ratings,random_state = 1, test_size=0.1)

In [77]:
print(train.shape)
print(test.shape)

(90803, 24)
(10090, 24)


## item profile

In [78]:
genres

,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
movieId,,,,,,,,,,,,,,,,,,,,
1,0,0,1,1,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0
2,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0
4,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0
5,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193581,0,1,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0
193583,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0
193585,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0


## User profile

In [80]:
gen_cols=genres.columns
gen_cols

Index(['(no genres listed)', 'Action', 'Adventure', 'Animation', 'Children',
       'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir',
       'Horror', 'IMAX', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller',
       'War', 'Western'],
      dtype='object')

In [84]:
for cols in gen_cols:
    train[cols] = train[cols] * train['rating']

In [87]:
user_profile=train.groupby('userId').mean()

In [88]:
user_profile

,movieId,rating,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,Documentary,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
userId,,,,,,,,,,,,,,,,,,,,,
1,1849.129187,4.358852,NaN,4.294118,4.371795,4.653846,4.555556,4.250000,4.384615,NaN,...,5.000000,3.562500,NaN,4.631579,4.062500,4.304348,4.189189,4.156863,4.523810,4.333333
2,71916.851852,3.962963,NaN,4.000000,4.500000,NaN,NaN,4.000000,3.812500,4.333333,...,NaN,3.000000,3.833333,NaN,4.000000,4.500000,3.875000,3.666667,4.500000,3.500000
3,7387.432432,2.418919,NaN,3.461538,2.500000,0.500000,0.500000,1.000000,0.500000,NaN,...,NaN,4.687500,NaN,0.500000,5.000000,0.500000,4.142857,4.000000,0.500000,NaN
4,1997.246231,3.572864,NaN,3.416667,3.655172,4.000000,3.750000,3.531250,3.750000,5.000000,...,4.000000,4.250000,NaN,4.000000,3.600000,3.442308,2.636364,3.571429,3.333333,3.800000
5,341.000000,3.589744,NaN,3.111111,3.285714,4.500000,4.142857,3.333333,3.727273,NaN,...,NaN,3.000000,4.000000,4.666667,4.000000,3.000000,2.500000,3.375000,3.333333,3.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
607,1824.043478,3.782609,NaN,3.709677,3.342105,3.333333,3.421053,3.361702,3.916667,NaN,...,NaN,4.129032,5.000000,3.600000,4.642857,3.615385,3.281250,4.148148,4.000000,4.000000
608,4369.306452,3.122984,NaN,3.314286,3.246875,3.088889,2.474026,2.721519,3.590551,3.000000,...,3.750000,3.311765,4.200000,2.722222,3.500000,2.860825,3.293103,3.515351,3.500000,2.700000
609,476.277778,3.250000,NaN,3.090909,3.200000,3.000000,3.000000,3.285714,3.400000,3.000000,...,NaN,3.500000,3.000000,NaN,NaN,3.200000,3.000000,3.230769,3.500000,4.000000


## My user profile

In [89]:
user_profile.loc[1111]

movieId               122634.943396
rating                     4.320755
(no genres listed)         4.500000
Action                     4.433333
Adventure                  4.580000
Animation                  4.142857
Children                   4.600000
Comedy                     4.555556
Crime                      4.000000
Documentary                     NaN
Drama                      4.029412
Fantasy                    3.833333
Film-Noir                       NaN
Horror                          NaN
IMAX                       4.291667
Musical                    4.000000
Mystery                    4.250000
Romance                    3.812500
Sci-Fi                     4.464286
Thriller                   4.272727
War                        4.333333
Western                         NaN
Name: 1111, dtype: float64

## predict

In [91]:
test.sample(1)['userId']

70520    449
Name: userId, dtype: int64

In [94]:
test[test['userId'] == 449]

,userId,movieId,rating,timestamp,(no genres listed),Action,Adventure,Animation,Children,Comedy,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
70548,449,3950,4.0,2003-05-18 04:37:09,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
70520,449,50,4.5,2003-05-18 04:32:39,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1.0,NaN,NaN
70530,449,1917,1.5,2003-05-18 04:15:22,NaN,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,NaN,NaN
70544,449,2858,4.0,2003-05-18 04:37:06,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN
70554,449,4973,4.0,2003-05-18 04:32:52,NaN,NaN,NaN,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN
70551,449,4027,3.5,2003-05-18 04:37:01,NaN,NaN,1.0,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
70539,449,2502,4.5,2003-05-18 04:37:03,NaN,NaN,NaN,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
70533,449,2012,2.5,2003-05-18 04:15:54,NaN,NaN,1.0,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1.0


userid 449 는 movieid 3950번에 몇점을 줄 것인가?

In [99]:
user_profile.loc[449]

movieId               2338.833333
rating                   3.216667
(no genres listed)            NaN
Action                   3.222222
Adventure                2.600000
Animation                     NaN
Children                 2.000000
Comedy                   2.500000
Crime                    3.666667
Documentary              4.500000
Drama                    3.750000
Fantasy                  2.166667
Film-Noir                     NaN
Horror                   2.500000
IMAX                          NaN
Musical                       NaN
Mystery                  3.625000
Romance                  2.100000
Sci-Fi                   2.833333
Thriller                 3.666667
War                      3.500000
Western                       NaN
Name: 449, dtype: float64

In [109]:
#4.0
(genres.loc[3950] * user_profile.loc[449]).mean()

3.75

In [110]:
#4.5
(genres.loc[50] * user_profile.loc[449]).mean()

3.6527777777777772

In [111]:
#1.5
(genres.loc[1917] * user_profile.loc[449]).mean()

2.9555555555555557

## 전체 데이터로 확장

In [124]:
from tqdm import tqdm_notebook

In [126]:
predict = []
for idx, row in tqdm_notebook(test.iterrows()):
    user=row['userId']
    predict.append((user_profile.loc[user] * row[gen_cols]).mean())

In [129]:
test['predict'] = predict

In [131]:
test.loc[test['predict'].isnull(),'predict'] = train['rating'].mean()

## Model validation

In [132]:
from sklearn.metrics import mean_squared_error

In [133]:
mse = mean_squared_error(test['rating'],test['predict'])
rmse = np.sqrt(mse)
rmse

0.921910154796705